# RLC resonance experiment
2022년 김현수, 명하진, 정수영, 최이선

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from tqdm import tqdm

import swimAD2 as ad2
import time

plt.rcParams.update({
    'figure.figsize': (4, 4),
    'figure.dpi': 144,
    'lines.markersize': 3,

    'axes.facecolor': 'white',
    'figure.facecolor': 'white'
})

ad2.disconnect()    # close all instances before connecting
alice = ad2.connect(0)       # connect to ad2

In [ ]:
def calculate_R(raw_data, regressed):
    sse = np.sum(np.square(raw_data - regressed))
    sst = np.sum(np.square(raw_data - raw_data.mean()))
    return 1 - sse/sst

def fit(f, xdata, ydata, plot=False, **kwargs):

    xdata = np.array(xdata).flatten()
    ydata = np.array(ydata).flatten()
    
    if plot:
        plt.plot(xdata, ydata, 'k.')
        if 'sigma' in kwargs:
            plt.errorbar(xdata, ydata, kwargs['sigma'], fmt='none', c='k')

    popt, pcov = curve_fit(f, xdata, ydata, **kwargs)
    perr = np.sqrt(np.diag(pcov))

    R = calculate_R(ydata, f(xdata, *popt))

    if plot:
        xlim, ylim = plt.xlim(), plt.ylim()
        plt.xlim(*xlim)
        plt.ylim(*ylim)
        x = np.linspace(*xlim, 100)
        plt.plot(x, f(x, *popt), 'r--')
        plt.show()

    return popt, perr, R

In [ ]:
# resonance amplitude ratio
def D(w, w_0, A, gamma):
    return A * w / np.sqrt((w_0*w_0 - w*w)*(w_0*w_0 - w*w) + (gamma*w)*(gamma*w))

# phase difference
def delta(w, w_0, gamma):
    return np.arctan(gamma * w / (w_0**2 - w**2)) % np.pi - np.pi / 2

In [ ]:
frequencies = np.linspace(150e3, 250e3, 50)

In [ ]:
DATA = []

for f in tqdm(frequencies):

    ad2.config_output(alice, frequency=f, amplitude=1, offset=0, channel=0)
    ad2.start_output(alice, channel=0)

    time.sleep(0.05)

    ad2.config_input(alice, range1=3, range2=3, sample_rate=512*f)
    
    t, v1, v2 = ad2.measure(alice)

    ad2.stop_output(alice, channel=-1)
        
    nv1, nv2 = v1 - v1.mean(), v2 - v2.mean()
    data = np.array([t, v1, v2, nv1, nv2])

    DATA.append(data)
    
    ad2.reset_output(alice)

DATA = np.array(DATA)


ratios, phases = [], []

for data in DATA:
    t, v1, v2, nv1, nv2 = data
    
    v1_amplitude = np.sqrt(2) * (nv1**2).mean()**(1/2)
    v2_amplitude = np.sqrt(2) * (nv2**2).mean()**(1/2)
    ratio = v2_amplitude / v1_amplitude
    ratios.append(ratio)

    product = nv1 * nv2
    product_amplitude = v1_amplitude * v2_amplitude / 2
    cos_phase = product.mean() / product_amplitude
    phase = np.arccos(cos_phase if abs(cos_phase) < 1 else 1)
    phases.append(phase)

In [ ]:
freq_coeff, freq_err, freq_R2 = fit(D, frequencies, ratios, plot=True, p0=[frequencies[np.argmax(ratios)], 1, 1])

ratio_coeff, ratio_err, ratio_R2 = fit(delta, frequencies, phases, plot=True, p0=[freq_coeff[0], freq_coeff[2]])